# Тюториал 4. Введение.  Классификация текстов 

Цель данного тюториала -- познакомить участников с базовым вариантом использованием **DeepPavlov**.

## Документация

Документация по моделям и коду: http://docs.deeppavlov.ai

Для бизнеса: http://ipavlov.ai/

## Установка зависимостей и библиотеки

Начнем с установки библиотеки `DeepPavlov`:
```
pip install deeppavlov
```
а также дополнительных зависимостей для использования нейросетевой модели классификации на `Keras` с `TensorFlow` бэкендом.
```
python -m deeppavlov install <config_path>
```
где `<config_path>` - путь к файлу-конфигу выбранной модели (например, `deeppavlov/configs/classifiers/intents_snips.json`) или просто название файла без расширения (например, `intents_snips`).

## Скачивание

Чтобы использовать модель для получения предсказаний в командной строке, запустите:
```
python -m deeppavlov interact <config_path> [-d]
```
где флаг ``-d`` скачивает необходимые данные -- файлы предобученной модели, эмбеддинги, файлы датасета (optional).

Натренировать модель можно следующим образом:
```
python -m deeppavlov train <config_path> [-d]
```
Общий список возможных режимов запуска следующий:
```
python -m deeppavlov <action> <config_path> [-d]
```
* ``<action>`` может принимать следующие занчения:
    * ``download`` -- скачать данные модели (то же самое, что указать для любой другой команды флаг ``-d``),
    * ``train`` -- натренировать модель на данных, указанных в конфиге,
    * ``evaluate`` -- посчитать значения метрик на исходном датасете,
    * ``interact`` -- вызов пайплайна на предикт в командной строке,
    * ``riseapi`` -- поднять REST API сервер (см. [docs](http://docs.deeppavlov.ai/en/master/integrations/rest_api.html)),
    * ``interactbot`` -- поднять пайплайн как Telegram bot (см. [docs](http://docs.deeppavlov.ai/en/master/integrations/telegram.html)),
    * ``interactmsbot`` -- поднять пайплайн как Miscrosoft Bot Framework сервер (см. [docs](http://docs.deeppavlov.ai/en/master/integrations/ms_bot.html)),
    * ``predict`` -- получить предикты для прмиеров из `stdin` или из `<file_path>`, если ``-f <file_path>`` указан.
* ``<config_path>`` -- полный путь или название файла-конфига.
* ``-d`` скачивает необходимые данные.
 

## Конфигурационный файл

In [11]:
import json

from deeppavlov import configs

config_path = configs.classifiers.rusentiment_elmo_twitter_cnn
print(type(config_path), config_path)

with open(config_path, "r") as f:
    config = json.load(f)

print(json.dumps(config, indent=2))

<class 'pathlib.PosixPath'> /home/dilyara/Documents/GitHub/DeepPavlov/deeppavlov/configs/classifiers/rusentiment_elmo_twitter_cnn.json
{
  "dataset_reader": {
    "class_name": "basic_classification_reader",
    "x": "text",
    "y": "label",
    "data_path": "{DOWNLOADS_PATH}/rusentiment/",
    "train": "rusentiment_random_posts.csv",
    "test": "rusentiment_test.csv"
  },
  "dataset_iterator": {
    "class_name": "basic_classification_iterator",
    "seed": 42,
    "field_to_split": "train",
    "split_seed": 23,
    "split_fields": [
      "train",
      "valid"
    ],
    "split_proportions": [
      0.9,
      0.1
    ]
  },
  "chainer": {
    "in": [
      "x"
    ],
    "in_y": [
      "y"
    ],
    "pipe": [
      {
        "id": "classes_vocab",
        "class_name": "simple_vocab",
        "fit_on": [
          "y"
        ],
        "save_path": "{MODEL_PATH}/classes.dict",
        "load_path": "{MODEL_PATH}/classes.dict",
        "in": "y",
        "out": "y_ids"
      },

## Dataset Reader

DatasetReader - компонента библиотеки для чтения файлов. `DeepPavlov` содержит несколько различных DatasetReaders, пользователи могут как использовать уже готовый DatasetReader, так и написать свою компоненту для чтения данных, однако стоит учитывать, что данные на выходе должны быть в определенном формате. 

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/dataset_readers.html

In [12]:
print(json.dumps(config["dataset_reader"], indent=2))

{
  "class_name": "basic_classification_reader",
  "x": "text",
  "y": "label",
  "data_path": "{DOWNLOADS_PATH}/rusentiment/",
  "train": "rusentiment_random_posts.csv",
  "test": "rusentiment_test.csv"
}


## Dataset Iterator

DatasetIterator - компонента библиотеки для итерирования по датасету (создания батчей, получения всех примеров). `DeepPavlov` содержит несколько разных DatasetIterators, пользователи могут как использовать уже готовый DatasetIterator, так и написать свою компоненту для итерирования по данным.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/dataset_iterators.html

In [13]:
print(json.dumps(config["dataset_iterator"], indent=2))

{
  "class_name": "basic_classification_iterator",
  "seed": 42,
  "field_to_split": "train",
  "split_seed": 23,
  "split_fields": [
    "train",
    "valid"
  ],
  "split_proportions": [
    0.9,
    0.1
  ]
}


## Preprocessor

Предобработать текст можно, используя различные компоненты либо написав свою.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/preprocessors.html

In [16]:
print(json.dumps(config["chainer"]["pipe"][1], indent=2))

{
  "in": [
    "x"
  ],
  "out": [
    "x_prep"
  ],
  "class_name": "dirty_comments_preprocessor",
  "remove_punctuation": false
}


## Tokenizer

Для использования потокенных (пословных) векторных представлений необходимо разделить предобработанный текст на токены (слов и пунктуационные символы).
`DeepPavlov` содержит несколько различных токенизаторов. 

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/tokenizers.html

In [17]:
print(json.dumps(config["chainer"]["pipe"][2], indent=2))

{
  "in": "x_prep",
  "out": "x_tok",
  "id": "my_tokenizer",
  "class_name": "nltk_tokenizer",
  "tokenizer": "wordpunct_tokenize"
}


## Embedder

В данном тюториале используем недообучаемые векторные представления ELMo. 
Все компоненты для получения векторных представлений текстов унифицированы, поэтому пользователи могут использовать любые векторные представления из предложенных в документации: GloVe, fastText, ELMo.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/embedders.html

In [18]:
print(json.dumps(config["chainer"]["pipe"][3], indent=2))

{
  "in": [
    "x_tok"
  ],
  "out": [
    "x_emb"
  ],
  "id": "my_embedder",
  "class_name": "elmo_embedder",
  "elmo_output_names": [
    "elmo"
  ],
  "mini_batch_size": 32,
  "spec": "http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-twitter_2013-01_2018-04_600k_steps.tar.gz",
  "pad_zero": true
}


## Vocabulary of classes

По умолчанию каждый класс считывается и хранится в виде строки.
Поэтому мы должны просмотреть все встречающиеся в датасете классы, собрать по ним словарь, а затем преобразовать классы в индексы.

Нейросетевые классификаторы принимают one-hot представление распределения по классам, а возвращают распределение вероятностей принадлежности классам.
Чтобы получить one-hot представление классов, необходимо также преобразовать полученные из словаря индексы классов.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/core/data.html

In [19]:
print(json.dumps(config["chainer"]["pipe"][0], indent=2))

{
  "id": "classes_vocab",
  "class_name": "simple_vocab",
  "fit_on": [
    "y"
  ],
  "save_path": "{MODEL_PATH}/classes.dict",
  "load_path": "{MODEL_PATH}/classes.dict",
  "in": "y",
  "out": "y_ids"
}


**One-hotter**

Компонента для преобразования индексов в one-hot представление.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/preprocessors.html

In [20]:
print(json.dumps(config["chainer"]["pipe"][4], indent=2))

{
  "in": "y_ids",
  "out": "y_onehot",
  "class_name": "one_hotter",
  "depth": "#classes_vocab.len",
  "single_vector": true
}


**Converting from probability to labels**

Так как нейросетевые модели возвращают вероятности распределения по классам, поэтому нам необходимо также преобразовать вектора вероятностей в классы.
Для этого используем `Proba2Labels` компоненту для преобразования вероятнсотей в индексы, а далее словарь по классам для преобразования индексов в текстовые назвнаия классов.

`Proba2Labels` компонента имеет несколько режимов:
* если `max_proba`=true, возвращает индексы с наибольшей вероятностью,
* если задано значение `confident_threshold` (от 0 до 1), возвращает индексы, вероятность принадлежности к классам которых выше заданного порога,
* если задано целочисленное значение `top_n`, возвращает `top_n` индексов с наибольшими вероятностями.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/preprocessors.html

In [21]:
print(json.dumps(config["chainer"]["pipe"][6], indent=2))

{
  "in": "y_pred_probas",
  "out": "y_pred_ids",
  "class_name": "proba2labels",
  "max_proba": true
}


## Classifier

`DeepPavlov` содержит несколько различных моделей для классификации текстов: классификаторы `sklearn`, нейросетевые модели на `Keras` с `TensorFlow` бэкендом, классификатор на основе BERT  архитектуры на `TensorFlow`.
Данный тюториал демонстрирует, как построить нейросетевой классификатор неглубокой широкой свёрточной сети (shallow and wide Convolutional Neural Network) на `Keras`. 

`KerasClassificationModel` - класс-компонента, строящий `Keras` классификатор, а архитектура сети задается в отдельном методе класса.

**DOCS:** http://docs.deeppavlov.ai/en/latest/apiref/models/classifiers.html

In [22]:
print(json.dumps(config["chainer"]["pipe"][5], indent=2))

{
  "in": [
    "x_emb"
  ],
  "in_y": [
    "y_onehot"
  ],
  "out": [
    "y_pred_probas"
  ],
  "main": true,
  "class_name": "keras_classification_model",
  "save_path": "{MODEL_PATH}/model",
  "load_path": "{MODEL_PATH}/model",
  "embedding_size": "#my_embedder.dim",
  "n_classes": "#classes_vocab.len",
  "kernel_sizes_cnn": [
    3,
    5,
    7
  ],
  "filters_cnn": 256,
  "optimizer": "Adam",
  "learning_rate": 0.01,
  "learning_rate_decay": 0.1,
  "loss": "categorical_crossentropy",
  "last_layer_activation": "softmax",
  "coef_reg_cnn": 0.001,
  "coef_reg_den": 0.01,
  "dropout_rate": 0.5,
  "dense_size": 100,
  "model_name": "cnn_model"
}


## Параметры тренировки

In [23]:
print(json.dumps(config["train"], indent=2))

{
  "epochs": 100,
  "batch_size": 128,
  "metrics": [
    "f1_weighted",
    "f1_macro",
    "sets_accuracy",
    {
      "name": "roc_auc",
      "inputs": [
        "y_onehot",
        "y_pred_probas"
      ]
    }
  ],
  "validation_patience": 5,
  "val_every_n_epochs": 1,
  "log_every_n_epochs": 1,
  "show_examples": false,
  "evaluation_targets": [
    "train",
    "valid",
    "test"
  ],
  "tensorboard_log_dir": "{MODEL_PATH}/logs",
  "class_name": "nn_trainer"
}


## Metadata

In [24]:
print(json.dumps(config["metadata"], indent=2))

{
  "variables": {
    "ROOT_PATH": "~/.deeppavlov",
    "DOWNLOADS_PATH": "{ROOT_PATH}/downloads",
    "MODELS_PATH": "{ROOT_PATH}/models",
    "MODEL_PATH": "{MODELS_PATH}/classifiers/rusentiment_v10"
  },
  "requirements": [
    "{DEEPPAVLOV_PATH}/requirements/tf.txt",
    "{DEEPPAVLOV_PATH}/requirements/tf-hub.txt"
  ],
  "labels": {
    "telegram_utils": "IntentModel",
    "server_utils": "KerasIntentModel"
  },
  "download": [
    {
      "url": "http://files.deeppavlov.ai/deeppavlov_data/classifiers/rusentiment_v10.tar.gz",
      "subdir": "{MODELS_PATH}/classifiers"
    }
  ]
}
